# This notebook contains the unfinished, work-in-progress methods. To access the working, current evolution of the baseline check out baseline.ipynb

Libraries

In [203]:
import torch 
from transformers import AutoTokenizer, AutoModelForTokenClassification
import os
import pandas as pd
import tensorflow as tf
from datasets import Dataset

Getting the model

In [2]:
MODEL_NAME = 'bert-base-uncased'


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [3]:
data_path = os.path.join('..','data', 'raw', 'train.json')

data = pd.read_json(data_path)

data_dataset = Dataset.from_pandas(data)
#data_dataset = data_dataset.remove_columns(['full_text','trailing_whitespace', 'document'])

In [204]:
LABELS_LIST = ['B-NAME_STUDENT', 'B-EMAIL', 'B-USERNAME', 'B-ID_NUM', 'B-PHONE_NUM', 'B-URL_PERSONAL', 'B-STREET_ADDRESS', 'I-NAME_STUDENT', 'I-EMAIL', 'I-USERNAME', 'I-ID_NUM', 'I-PHONE_NUM','I-URL_PERSONAL','I-STREET_ADDRESS', 'O']

label2id = {label: i for i, label in enumerate(LABELS_LIST)}
label2id['[PAD]'] = -100
id2label = {i: label for label, i in label2id.items()}
id2label

{0: 'B-NAME_STUDENT',
 1: 'B-EMAIL',
 2: 'B-USERNAME',
 3: 'B-ID_NUM',
 4: 'B-PHONE_NUM',
 5: 'B-URL_PERSONAL',
 6: 'B-STREET_ADDRESS',
 7: 'I-NAME_STUDENT',
 8: 'I-EMAIL',
 9: 'I-USERNAME',
 10: 'I-ID_NUM',
 11: 'I-PHONE_NUM',
 12: 'I-URL_PERSONAL',
 13: 'I-STREET_ADDRESS',
 14: 'O',
 -100: '[PAD]'}

## Preprocessing - version 1

### Label encodings

In [ ]:
def encode_labels(example):
    """
    to be used with datasets.map() with batched=False
    
    Encodes the labels into integers.
    
    """
    labels = example['labels']
    encoded = [label2id[label] for label in labels]
    return {'labels': encoded}

In [206]:
data_labels_encoded = data_dataset.map(encode_labels)

Map:   0%|          | 0/6807 [00:00<?, ? examples/s]

KeyboardInterrupt: 

### Encoding tokens

In [ ]:
def encode_tokens(examples):
    """
    o be used with datasets.map() with batched=True

    Expects a batch of examples to be tokenized and returns the tokenized examples.
    
    Warning: Doesn't align labels with tokens. If the text is turned into chunks, each chunk will have the labels for the full text in it's row.
    """
    result = tokenizer(examples["full_text"], 
                       padding="max_length",
                       max_length=512, 
                       truncation=True, 
                       return_overflowing_tokens=True, 
                       return_offsets_mapping=True, 
                       stride=10, 
                       return_tensors='pt')
    
    sample_map = result.pop('overflow_to_sample_mapping')
    result['overflow_to_sample_mapping'] = sample_map
    print(sample_map)
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result

tokenized_dataset = data_labels_encoded.map(encode_tokens, batched=True)

Map:   0%|          | 0/6807 [00:00<?, ? examples/s]

tensor([  0,   0,   1,  ..., 998, 999, 999])
tensor([  0,   1,   2,  ..., 998, 999, 999])
tensor([  0,   0,   1,  ..., 998, 999, 999])
tensor([  0,   0,   1,  ..., 998, 999, 999])
tensor([  0,   0,   0,  ..., 998, 999, 999])
tensor([  0,   0,   1,  ..., 998, 998, 999])
tensor([  0,   0,   1,  ..., 805, 806, 806])


In [221]:
import numpy as np

def tokenize_and_align(example, sub_word_labeling = True, overlap_size = 0):
    """
    To be used with datasets.map() with batched=False

    Takes in 
        - example : an example from the datasets class
        - sub_word_labeling: if True, labels are assigned to subwords of words. If False, subwords (other than the 1st one) are assigned the label -100
        - overlap_size: the number of tokens that overlap between two consecutive chunks
        
    outputs:
        - a Dict[]->List with columns:
            - of the bert tokenizer output
            - encoded labels
    """

    org_labels = example['labels']
    tokenized_inputs = tokenizer(example['tokens'], is_split_into_words=True, return_offsets_mapping=True, truncation=True, padding='max_length', max_length=512, return_overflowing_tokens=True, stride=overlap_size, return_tensors='pt')
    new_labels = []
    # Update the labels with the new tokenization

    it = 0
    #setting prev_offset to a value that will never be reached
    #it is set here, to avoid the case where the first token is a mid-subword
    prev_offset = (1,1)

    for doc_offset in tokenized_inputs['offset_mapping']:
        aligned_labels = []
        
        
        arr_offset = np.array(doc_offset)
        #making a list of -100 of the same length as the number of BERT tokens in the chunk
        arr_aligned_labels = np.ones(len(doc_offset), dtype=int) * (-100)

        # calculating the number of labels present in the chunk
        nb_labels_in_chunk = len(arr_offset[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)])

        # assigning the labels to the corresponding tokens, meaning tokens with offset (0,y)
        arr_aligned_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = org_labels[:nb_labels_in_chunk]

        # removing the labels that have been assigned to the chunk from the list of labels
        org_labels = org_labels[nb_labels_in_chunk:]

        
        if sub_word_labeling:
            #iterate over tokens where x!=0 and fill them in 
            subword_idxs = np.where(arr_offset[:,0] != 0)[0]
            arr_subword_labels = arr_aligned_labels[arr_offset[:,0] != 0]
            print(subword_idxs)
            print(arr_aligned_labels[474])
            print(len(arr_subword_labels) == len(subword_idxs))
            for idx, internal_label in  zip(subword_idxs, arr_subword_labels):
                arr_aligned_labels[idx] = arr_aligned_labels[idx-1]
            print('aligned labels: ',arr_aligned_labels[subword_idxs])

        new_labels.append(arr_aligned_labels)


    tokenized_inputs["labels"] = new_labels
    tokenized_inputs.pop("offset_mapping")
    tokenized_inputs.pop("overflow_to_sample_mapping")

    return tokenized_inputs
    
data_small = data_labels_encoded.select(range(2))
data_small = data_small.map(tokenize_and_align, batched=False)

print(data_small['labels'][0][0][474])

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

[  6   9  13  14  16  55  57  60  62  67  68  72  73  80  81  91  92 113
 121 150 155 156 173 215 216 217 221 222 229 264 265 437 441 445 468 471
 475 476 478 495 509 510]
14
True
aligned labels:  [14 14  0  0  7 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14]
[  1  20  69 127 140 141 152 153 154 218 221 225 226 228]
-100
True
aligned labels:  [-100   14   14   14   14   14   14   14   14   14   14   14   14   14]
[  7  43  69  91  92 194 199 255 275 305 345 385 404 432 462 463 464]
14
True
aligned labels:  [14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14]
[]
-100
True
aligned labels:  []
14


In [ ]:

data_chunked_encoded = data_labels_encoded.map(lambda x: tokenize_and_align(x,sub_word_labeling=True), batched=False)
#time to tokenize and chunk WITH subword labeling : 1m58s

In [ ]:
print("labels aligned with tokens, with subword labeling")
print(tokenizer.convert_ids_to_tokens(data_chunked_encoded['input_ids'][0][0][2:14]))
print(data_chunked_encoded['labels'][0][0][2:14])

labels aligned with tokens, with subword labeling
['thinking', 'for', 'innovation', 'reflex', '##ion', '-', 'av', '##ril', '2021', '-', 'nat', '##hal']
[14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 0, 0]


In [ ]:
data_chunked_encoded_sub_labeling_false = data_labels_encoded.map(lambda x: tokenize_and_align(x,sub_word_labeling=False), batched=False)
#time to tokenize and chunk WITHOUT subword labeling : 46s

Map:   0%|          | 0/6807 [00:00<?, ? examples/s]

In [ ]:
print("labels aligned with tokens, without subword labeling")
print(tokenizer.convert_ids_to_tokens(data_chunked_encoded_sub_labeling_false['input_ids'][0][0][2:14]))
print(data_chunked_encoded_sub_labeling_false['labels'][0][0][2:14])

labels aligned with tokens, without subword labeling
['thinking', 'for', 'innovation', 'reflex', '##ion', '-', 'av', '##ril', '2021', '-', 'nat', '##hal']
[14, 14, 14, 14, -100, 14, 14, -100, 14, 14, 0, -100]


### Flattening the rows

In [ ]:
from functools import reduce

#flatten the dataset
data_flat = {}

keys = ['labels', 'input_ids', 'token_type_ids', 'attention_mask']

for key in keys:
    data_flat[key] = reduce(lambda x,y: x+y, data_chunked_encoded[key])

data_flat = Dataset.from_dict(data_flat)

In [ ]:
#checking if each row has length 512 for each column
print("Checking if each row has length 512 for each column")
for example in data_flat:
    for key in keys:
        if len(example[key]) != 512:
            print(f"Error in {key}, length is {len(example[key])} instead of 512")
            break
print("All good")

Checking if each row has length 512 for each column
All good


In [70]:
len(data_flat['input_ids'][0])

512

## Preprocessing - version 0

In [222]:
def tokenize_and_preserve_labels(examples):
    labels = []
    tokenized_sentence = []
    for word, label in zip(examples['tokens'], examples['labels']):
        #tokenizes the word using BERT's subword tokenizer
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)
        tokenized_sentence.extend(tokenized_word)
        #adds the same label to all the subwords of the word
        labels.extend([label] * n_subwords)
    examples['tokens'] = tokenized_sentence
    examples['labels'] = labels
    return examples

In [223]:
tokenized_data = data_dataset.map(tokenize_and_preserve_labels,batched=False)

Map:   0%|          | 0/6807 [00:00<?, ? examples/s]

In [224]:
def chunk(examples, block_size=510, sliding_window=False):
    tokenized_sentences = []
    labels = []
    for i in range(0, len(examples['tokens']), block_size):
        chunk_token = examples['tokens'][i:i+block_size]
        chunk_label = examples['labels'][i:i+block_size]
        if len(chunk_token) < block_size:
            chunk_token += ['[PAD]'] * (block_size - len(chunk_token))
            chunk_label += ['[PAD]'] * (block_size - len(chunk_label))
        tokenized_sentences.append(chunk_token)
        labels.append(chunk_label)
    return {'tokens': tokenized_sentences, 'labels': labels}

In [225]:
chunked_data = tokenized_data.map(chunk, batched=False)
chunked_data = chunked_data.remove_columns(['document', 'full_text','trailing_whitespace'])

Map:   0%|          | 0/6807 [00:00<?, ? examples/s]

In [226]:
from functools import reduce
flat_chunks_tokens = list(reduce(lambda x, y: x + y, chunked_data['tokens'], []))
flat_chunks_labels = list(reduce(lambda x, y: x + y, chunked_data['labels'], []))

In [227]:
chunked_flattened_data = Dataset.from_dict({'tokens': flat_chunks_tokens, 'labels': flat_chunks_labels})

Setting the '[PAD]' label to be encoded as -100 so that it's ignored by the loss function

In [228]:
chunked_flattened_data

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 12812
})

In [229]:
def encode_tokens(example):
    detokenized = list(map(lambda x: ' '.join(x), example['tokens']))
    detokenized = list(map(lambda x: x.replace(' ##', ''), detokenized))
    encoded = tokenizer(detokenized, truncation=True, is_split_into_words = False, return_tensors='pt')
    return encoded

In [230]:
encoded_data = chunked_flattened_data.map(encode_tokens, batched=True)

Map:   0%|          | 0/12812 [00:00<?, ? examples/s]

In [137]:
def encode_labels(example):
    labels = example['labels']
    #adding -100 for the [CLS] token and [SEP] token
    encoded = [-100] + [label2id[label] for label in labels] + [-100]
    return {'labels': encoded}

encoded_labels = encoded_data.map(encode_labels, batched=False)

Map:   0%|          | 0/12812 [00:00<?, ? examples/s]

In [138]:
encoded_data

Dataset({
    features: ['tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 12812
})

In [140]:
len(encoded_labels['tokens'][0]), len(encoded_labels['labels'][0]), len(encoded_labels['input_ids'][0])

(510, 512, 512)

In [145]:
encoded_data_split = encoded_labels.train_test_split(test_size=0.1)

In [142]:
tokenizer.decode(encoded_labels['input_ids'][0][11:20])

'- nathalie sylla challenge & selection'

In [144]:
encoded_data_split

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 11530
    })
    test: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1282
    })
})

## Model training & evaluation

In [170]:
#encoded_data_split = data_flat.train_test_split(test_size=0.1)

In [146]:
import evaluate

seqeval = evaluate.load("seqeval")

In [147]:
import numpy as np


def compute_metrics(p):
    
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [LABELS_LIST[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [LABELS_LIST[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [94]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME, num_labels=len(id2label), id2label=id2label, label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [148]:
#freezing the BERT layers
for param in model.base_model.parameters():
    param.requires_grad = False

In [149]:
# training_args = TrainingArguments(
#     output_dir="model_baseline_v0.1",
#     learning_rate=2e-3,
#     per_device_train_batch_size=20,
#     per_device_eval_batch_size=20,
#     num_train_epochs=2,
#     weight_decay=0.01,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
# )


target_dir = "model/trainer_model_initial_preprocessing"

training_args = TrainingArguments(output_dir=target_dir, evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_data_split["train"],
    eval_dataset=encoded_data_split["test"],
    #tokenizer=tokenizer,
    #data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/homebrew/anaconda3/envs/projet-en-tal/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/4326 [00:00<?, ?it/s]

{'loss': 0.0005, 'learning_rate': 4.4220989366620436e-05, 'epoch': 0.35}
{'loss': 0.0005, 'learning_rate': 3.844197873324087e-05, 'epoch': 0.69}


  0%|          | 0/161 [00:00<?, ?it/s]

/opt/homebrew/anaconda3/envs/projet-en-tal/lib/python3.11/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

{'eval_loss': 0.0002911491028498858, 'eval_precision': 0.9073634204275535, 'eval_recall': 0.9646464646464646, 'eval_f1': 0.9351285189718482, 'eval_accuracy': 0.9998976614599324, 'eval_runtime': 53.6274, 'eval_samples_per_second': 23.906, 'eval_steps_per_second': 3.002, 'epoch': 2.0}
{'loss': 0.0005, 'learning_rate': 1.5325936199722607e-05, 'epoch': 2.08}
{'loss': 0.0005, 'learning_rate': 9.546925566343042e-06, 'epoch': 2.43}
{'loss': 0.0005, 'learning_rate': 3.7679149329634766e-06, 'epoch': 2.77}


  0%|          | 0/161 [00:00<?, ?it/s]

/opt/homebrew/anaconda3/envs/projet-en-tal/lib/python3.11/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.0002923872380051762, 'eval_precision': 0.9073634204275535, 'eval_recall': 0.9646464646464646, 'eval_f1': 0.9351285189718482, 'eval_accuracy': 0.9998976614599324, 'eval_runtime': 53.2128, 'eval_samples_per_second': 24.092, 'eval_steps_per_second': 3.026, 'epoch': 3.0}
{'train_runtime': 1946.9332, 'train_samples_per_second': 17.766, 'train_steps_per_second': 2.222, 'train_loss': 0.0004889280520210759, 'epoch': 3.0}


TrainOutput(global_step=4326, training_loss=0.0004889280520210759, metrics={'train_runtime': 1946.9332, 'train_samples_per_second': 17.766, 'train_steps_per_second': 2.222, 'train_loss': 0.0004889280520210759, 'epoch': 3.0})

In [150]:
trainer.save_model('model/model_initial_preprocessing')

In [183]:
#put trainer on cpu

trainer.model.to('mps')

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [184]:
trainer.evaluate()

  0%|          | 0/161 [00:00<?, ?it/s]

/opt/homebrew/anaconda3/envs/projet-en-tal/lib/python3.11/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.0002923872380051762,
 'eval_precision': 0.9073634204275535,
 'eval_recall': 0.9646464646464646,
 'eval_f1': 0.9351285189718482,
 'eval_accuracy': 0.9998976614599324,
 'eval_runtime': 54.1181,
 'eval_samples_per_second': 23.689,
 'eval_steps_per_second': 2.975,
 'epoch': 3.0}

In [162]:
model = trainer.model

In [171]:
sen = 'my name is John Smith and my email is john.smith@gmail.com.'

model = model.to('cpu')

predictions = model(**tokenizer(sen, return_tensors='pt'))
predictions = np.argmax(predictions.logits.detach().numpy(), axis=2)

In [172]:
bert_tokens = tokenizer.tokenize(sen)
bert_tokens = ['[CLS]'] + bert_tokens + ['[SEP]']

In [176]:
predictions

array([[14, 14, 14, 14,  0,  7, 14, 14, 14, 14,  0,  1,  1,  1,  1,  1,
         1,  1, 14, 14]])

In [175]:
[(id2label[pred], token) for pred, token in zip(predictions[0], bert_tokens)]

[('O', '[CLS]'),
 ('O', 'my'),
 ('O', 'name'),
 ('O', 'is'),
 ('B-NAME_STUDENT', 'john'),
 ('I-NAME_STUDENT', 'smith'),
 ('O', 'and'),
 ('O', 'my'),
 ('O', 'email'),
 ('O', 'is'),
 ('B-NAME_STUDENT', 'john'),
 ('B-EMAIL', '.'),
 ('B-EMAIL', 'smith'),
 ('B-EMAIL', '@'),
 ('B-EMAIL', 'gma'),
 ('B-EMAIL', '##il'),
 ('B-EMAIL', '.'),
 ('B-EMAIL', 'com'),
 ('O', '.'),
 ('O', '[SEP]')]

In [177]:
#load the model into a pipeline and evaluaye it on the test set

from transformers import TokenClassificationPipeline

model_loaded = AutoModelForTokenClassification.from_pretrained('model/model_initial_preprocessing')
model_loaded = model.to('cpu')

In [180]:
essay = """
'The consequences of Great Planning', by John Smith

The consequences of great planning are many. One of the most important consequences is that it allows us to achieve our goals. When we plan, we are able to think about what we want to achieve and how we are going to achieve it. This helps us to focus on what is important and to avoid wasting time on things that are not important. Planning also helps us to see the big picture and to understand how all the different parts of a project fit together. This can help us to make better decisions and to avoid making mistakes.

Another consequence of great planning is that it helps us to be more efficient. When we plan, we can break a big task down into smaller tasks and then work on each task one at a time. This can help us to stay focused and to avoid feeling overwhelmed. Planning can also help us to identify potential problems before they occur and to come up with solutions to those problems. This can help us to save time and to avoid wasting resources.

Finally, great planning can help us to be more successful. When we plan, we are able to set clear goals and to create a roadmap for how we are going to achieve those goals. This can help us to stay motivated and to keep moving forward, even when things get tough. Planning can also help us to track our progress and to see how far we have come. This can help us to stay focused and to keep working towards our goals, even when we face obstacles.

In conclusion, the consequences of great planning are many. Planning can help us to achieve our goals, to be more efficient, and to be more successful. It can help us to focus on what is important, to avoid wasting time, and to make better decisions. It can help us to stay motivated, to track our progress, and to keep moving forward. In short, great planning is essential for success in any endeavor.

email: john.smith@bmail.com
phone: 123-456-7890

"""

predictions = model(**tokenizer(essay, return_tensors='pt'))
predictions = np.argmax(predictions.logits.detach().numpy(), axis=2)

bert_tokens = tokenizer.tokenize(essay)
bert_tokens = ['[CLS]'] + bert_tokens + ['[SEP]']

[(id2label[pred], token) for pred, token in zip(predictions[0], bert_tokens)]

[('O', '[CLS]'),
 ('O', "'"),
 ('O', 'the'),
 ('O', 'consequences'),
 ('O', 'of'),
 ('O', 'great'),
 ('O', 'planning'),
 ('O', "'"),
 ('O', ','),
 ('O', 'by'),
 ('B-NAME_STUDENT', 'john'),
 ('I-NAME_STUDENT', 'smith'),
 ('O', 'the'),
 ('O', 'consequences'),
 ('O', 'of'),
 ('O', 'great'),
 ('O', 'planning'),
 ('O', 'are'),
 ('O', 'many'),
 ('O', '.'),
 ('O', 'one'),
 ('O', 'of'),
 ('O', 'the'),
 ('O', 'most'),
 ('O', 'important'),
 ('O', 'consequences'),
 ('O', 'is'),
 ('O', 'that'),
 ('O', 'it'),
 ('O', 'allows'),
 ('O', 'us'),
 ('O', 'to'),
 ('O', 'achieve'),
 ('O', 'our'),
 ('O', 'goals'),
 ('O', '.'),
 ('O', 'when'),
 ('O', 'we'),
 ('O', 'plan'),
 ('O', ','),
 ('O', 'we'),
 ('O', 'are'),
 ('O', 'able'),
 ('O', 'to'),
 ('O', 'think'),
 ('O', 'about'),
 ('O', 'what'),
 ('O', 'we'),
 ('O', 'want'),
 ('O', 'to'),
 ('O', 'achieve'),
 ('O', 'and'),
 ('O', 'how'),
 ('O', 'we'),
 ('O', 'are'),
 ('O', 'going'),
 ('O', 'to'),
 ('O', 'achieve'),
 ('O', 'it'),
 ('O', '.'),
 ('O', 'this'),
 ('O'

In [119]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_data_split["train"],
    eval_dataset=encoded_data_split["test"],
    compute_metrics=compute_metrics,
)

trainer.evaluate()

  0%|          | 0/161 [00:00<?, ?it/s]

{'eval_loss': 0.010430662892758846,
 'eval_precision': 0.10179640718562874,
 'eval_recall': 0.22869955156950672,
 'eval_f1': 0.1408839779005525,
 'eval_accuracy': 0.9983660777738643,
 'eval_runtime': 56.1491,
 'eval_samples_per_second': 22.832,
 'eval_steps_per_second': 2.867}

In [185]:
data_flat['labels'][0]

[-100,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 0,
 0,
 0,
 7,
 7,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14

In [189]:
len(encoded_labels['labels'][0])

512

In [194]:
for i, (l1,l2) in enumerate(zip(data_flat['labels'][0], encoded_labels['labels'][0])):
    if l1 != l2:
        print(i, l1, l2)

print(tokenizer.convert_ids_to_tokens(encoded_labels['input_ids'][0][474:479]))
print(tokenizer.convert_ids_to_tokens(data_flat['input_ids'][0][474:479]))

data_flat['input_ids'][0] == encoded_labels['input_ids'][0]

474 14 0
475 14 0
476 14 0
477 14 7
478 14 7
['nat', '##hal', '##ie', 'sy', '##lla']
['nat', '##hal', '##ie', 'sy', '##lla']


True

In [190]:
len(data_flat['labels'][0])

512